<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Import-Packages" data-toc-modified-id="Import-Packages-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Import Packages</a></span></li><li><span><a href="#Import-Data" data-toc-modified-id="Import-Data-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Import Data</a></span></li><li><span><a href="#Missing-Data-and-Data-Types" data-toc-modified-id="Missing-Data-and-Data-Types-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Missing Data and Data Types</a></span><ul class="toc-item"><li><span><a href="#Checking-on-NaNs" data-toc-modified-id="Checking-on-NaNs-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Checking on NaNs</a></span></li><li><span><a href="#Fixing-DataTypes" data-toc-modified-id="Fixing-DataTypes-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Fixing DataTypes</a></span></li></ul></li><li><span><a href="#Column-Transformation" data-toc-modified-id="Column-Transformation-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Column Transformation</a></span><ul class="toc-item"><li><span><a href="#Game-Clock" data-toc-modified-id="Game-Clock-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Game Clock</a></span></li><li><span><a href="#Player-Height" data-toc-modified-id="Player-Height-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Player Height</a></span></li><li><span><a href="#Offensive-and-Defensive-Personnel" data-toc-modified-id="Offensive-and-Defensive-Personnel-4.3"><span class="toc-item-num">4.3&nbsp;&nbsp;</span>Offensive and Defensive Personnel</a></span><ul class="toc-item"><li><span><a href="#Parsing-Offensive-Personnel" data-toc-modified-id="Parsing-Offensive-Personnel-4.3.1"><span class="toc-item-num">4.3.1&nbsp;&nbsp;</span>Parsing Offensive Personnel</a></span></li><li><span><a href="#Parsing-Defensive-Personnel" data-toc-modified-id="Parsing-Defensive-Personnel-4.3.2"><span class="toc-item-num">4.3.2&nbsp;&nbsp;</span>Parsing Defensive Personnel</a></span></li><li><span><a href="#Adding-to-DataFrame" data-toc-modified-id="Adding-to-DataFrame-4.3.3"><span class="toc-item-num">4.3.3&nbsp;&nbsp;</span>Adding to DataFrame</a></span></li></ul></li></ul></li></ul></div>

# 2019 NFL Big Data Bowl - Part 1: Data Cleaning
This set of notebooks will go through a past Kaggle competition known as the NFL Big Data Bowl. The data bowl this time focuses on **running plays**, and specifically, how far a running play will go. We've all watched the games on television, and during plays, once the running back gets the ball, we sometimes think "Oh that play is going nowhere" or "That's a touchdown". How are we making these decisions? Presumably, a standard armchair spectator is looking at the blocking that is taking place in the offensive line. These sorts of cues are what we hope to capture when we analyze the running plays. 

The notebooks will be arranged into parts that will take us through the various analyses that I perform. With any project, the data cleaning will always come first, to make sure the data can be effectively analyzed. This includes getting rid of duplicates, dropping missing data, splitting columns, and fixing data types. After this comes exploratory data analysis (EDA).

## Import Packages
This cell contains all the packages that we plan to use in this notebook. The two common packages `numpy` and `pandas` are sufficient for the data cleaning we'll do. The `json` package will be used for storing the resulting data types of each column.

In [1]:
import numpy as np
import pandas as pd
import json

## Import Data
Our provided training data is in a file called train.csv. This has all the columns we would ever need.

In [2]:
allData = pd.read_csv('./Data/train.csv', parse_dates=[29, 30])
allData.head()

C:\Users\mughi\miniconda3\envs\NFL_BDB_19\lib\site-packages\IPython\core\interactiveshell.py:3062: DtypeWarning: Columns (47) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,GameId,PlayId,Team,X,Y,S,A,Dis,Orientation,Dir,...,Week,Stadium,Location,StadiumType,Turf,GameWeather,Temperature,Humidity,WindSpeed,WindDirection
0,2017090700,20170907000118,away,73.91,34.84,1.69,1.13,0.40,81.99,177.18,...,1,Gillette Stadium,"Foxborough, MA",Outdoor,Field Turf,Clear and warm,63.0,77.0,8,SW
1,2017090700,20170907000118,away,74.67,32.64,0.42,1.35,0.01,27.61,198.70,...,1,Gillette Stadium,"Foxborough, MA",Outdoor,Field Turf,Clear and warm,63.0,77.0,8,SW
2,2017090700,20170907000118,away,74.00,33.20,1.22,0.59,0.31,3.01,202.73,...,1,Gillette Stadium,"Foxborough, MA",Outdoor,Field Turf,Clear and warm,63.0,77.0,8,SW
3,2017090700,20170907000118,away,71.46,27.70,0.42,0.54,0.02,359.77,105.64,...,1,Gillette Stadium,"Foxborough, MA",Outdoor,Field Turf,Clear and warm,63.0,77.0,8,SW
4,2017090700,20170907000118,away,69.32,35.42,1.82,2.43,0.16,12.63,164.31,...,1,Gillette Stadium,"Foxborough, MA",Outdoor,Field Turf,Clear and warm,63.0,77.0,8,SW


In [3]:
print('Data Shape:', allData.shape)

Data Shape: (682154, 49)


There are a ton of variables and observations here. The information some of the columns provide might overlap with others. Due to the sheer immense number, [here](https://www.kaggle.com/c/nfl-big-data-bowl-2020/data) is an external link to description of what each column is, and the reference values for the positional information.

## Missing Data and Data Types
The first step in cleaning is to examine how much missing data we have. Depending on how important the column is, we can either drop any rows where that value is missing, or just simply drop the column if we are not going to use it. Because this came from a Kaggle competition, it's likely most of the values will already be present, but it doesn't hurt to double-check.

As for data types, `pandas` usually assigns the most memory to the integer and float values i.e. `int64`. Depending on the use-case, this may not be needed. Furthermore, it's common to have some variables as categorical, such as the team or ID.

### Checking on NaNs
To get a broad overview, we can count up the number of NaNs we have in each column, to see which ones are the biggest offenders.

In [4]:
nasums = allData.isna().sum()
for i, (name, value) in enumerate(nasums.items()):
    print('{:22}\t{}'.format(name, value), end='')
    if i % 4 == 3:
        print()
    else:
        print('\t', end='')

GameId                	0	PlayId                	0	Team                  	0	X                     	0
Y                     	0	S                     	0	A                     	0	Dis                   	0
Orientation           	23	Dir                   	28	NflId                 	0	DisplayName           	0
JerseyNumber          	0	Season                	0	YardLine              	0	Quarter               	0
GameClock             	0	PossessionTeam        	0	Down                  	0	Distance              	0
FieldPosition         	8602	HomeScoreBeforePlay   	0	VisitorScoreBeforePlay	0	NflIdRusher           	0
OffenseFormation      	88	OffensePersonnel      	0	DefendersInTheBox     	22	DefensePersonnel      	0
PlayDirection         	0	TimeHandoff           	0	TimeSnap              	0	Yards                 	0
PlayerHeight          	0	PlayerWeight          	0	PlayerBirthDate       	0	PlayerCollegeName     	0
Position              	0	HomeTeamAbbr          	0	VisitorTeamAbbr       	0	Week              

Looking at the stats, the data is mostly clean on the key points. The biggest offenders are columns pertaining to the weather during the game. We can always use more reliable external sources to obtain what the weather was that day, so these are not important to the data itself.

The **stadium type** tells us what the stadium environment is like. For example, is it outdoors/indoors, in a dome, have a retractable roof? The only place where this would effect our prediction, is if the stadium was outdoors, and it was raining pretty heavily, which would make players slip around (the 2011 "Monsoon Bowl" between the Jaguars and Panthers is a good example). However, this is a relative rarity that may happen 1 or 2 times a season, and so training in these specific conditions will not be productive. Additionally, we also have the stadium and turf type itself, so even if we were to include the weather, the model would ideally learn that in some stadiums the weather has no effect.

The next variable is **field position**. This tells us which side of the field the possession team is on. I would say this is heavily important, especially since this determines the distance you are from the endzone. Additionally, the **yard line** is listed as a number between 0-50, so we would absolutely need the side to fully determine where the ball is. Therefore, we will remove all rows where we don't have the field position.

The other columns with low amounts will also have their rows removed, as those as datapoins which are quite important with respect to the running game. For example, it's much easier to block with 3 tight ends instead of the base 1. Personnel packages also let defenses infer one way or the other.

In conclusion, we will flat out **drop** the weather and stadium type columns, and **remove the rows** where the field position, offensive personnel, and orientation are absent.

In [5]:
colsToDrop = ['StadiumType', 'GameWeather', 'Temperature', 'Humidity',
             'WindSpeed', 'WindDirection']
# Delete the columns in colsToDrop
# and the rows where there are NaNs
cutDownData = allData.drop(columns=colsToDrop).dropna()
print('Shape after dropping:', cutDownData.shape)

Shape after dropping: (673432, 43)


Looks like we lost about 9000 records, which isn't too bad, considering we have over 670 thousand records in total.

### Fixing DataTypes
Another thing we might need to fix are data type mismatches. For example, some of these columns are direct strings, such as the **Team** column. However, this column can only take one of two values, "home" or "away". Thus, it's better if we code this as a categorical variable. The same would be a true for a number of other categories. Additionally, the default number data type is 64 bit, of which none of our values are that precise, so we can also save a bit on memory.

In [6]:
integerCols = ['Season', 'YardLine', 'Quarter', 'Down', 'Distance', 
               'HomeScoreBeforePlay', 'VisitorScoreBeforePlay', 'DefendersInTheBox',
               'Yards', 'PlayerWeight', 'Week']
floatCols = ['X', 'Y', 'S', 'A', 'Dis', 'Orientation', 'Dir']
categoricalCols = ['GameId', 'PlayId', 'Team', 'NflId', 'JerseyNumber',
                  'PossessionTeam', 'FieldPosition', 'NflIdRusher',
                  'OffenseFormation', 'Position', 'HomeTeamAbbr',
                  'VisitorTeamAbbr', 'Stadium']
cutDownData[integerCols] = cutDownData[integerCols].astype('int32')
cutDownData[floatCols] = cutDownData[floatCols].astype('float32')
cutDownData[categoricalCols] = cutDownData[categoricalCols].astype('category')
cutDownData.dtypes

GameId                               category
PlayId                               category
Team                                 category
X                                     float32
Y                                     float32
S                                     float32
A                                     float32
Dis                                   float32
Orientation                           float32
Dir                                   float32
NflId                                category
DisplayName                            object
JerseyNumber                         category
Season                                  int32
YardLine                                int32
Quarter                                 int32
GameClock                              object
PossessionTeam                       category
Down                                    int32
Distance                                int32
FieldPosition                        category
HomeScoreBeforePlay               

We've taken care of all the data types that we could assign directly to the data. For the other columns, there needs to be some data munging and extraction to get them into a form that we can work with.

## Column Transformation
There are a couple columns in our dataset which require transformation of the data already present. This could involve splitting of the info into multiple columns, or directly changing the way the column is represented. Most of these are simple to do.

### Game Clock
Currently the **game clock** column shows what the game clock in the stadium shows during the play. In every game, the game clock shows the time remaining in the quarter, so it ranges from 15:00 (15 minutes) down to 00:00 (0 minutes). However, we don't want to represent it like this in the data because it looks like a 24-hour time. A game clock of 14:14 *does not mean* the time is 2:14 PM. It means there is 14 minutes and 14 seconds left in the quarter. 

How should we change the representation to help our model the most? Thinking about the actual game, there's an important way the game plays out: *At the end of the second quarter (first half), the possession resets itself. Either the other team gets the ball, or a rekick occurs and the same team gets the ball.* Thus, past the 2-minute warning, any running plays that are present will either be draws (like delayed handoffs), or designed runs towards the sidelines in an effort to stop the clock. Of course, runs can also happen close to the goal line but those happen anyways. Given this definition, it seems appropriate that **we should list the game clock as the time remaining until the half.** Thus, it will be in two 30-minute intervals. We can also add a **Half** column, which denotes which half it is.

In [7]:
# Use str to get access to string methods.
# We split on the colon, and add it
# as a column. Then we create a sub
# data frame with the list data and
# add it to our data...
cutDownData['splittedGameClock'] = cutDownData.GameClock.str.split(':')
cutDownData[['GameClockMinute', 'GameClockSecond', 'GCms']] = pd.DataFrame(cutDownData.splittedGameClock.tolist(),
                                                                   index=cutDownData.index)
# Drop the extra column and convert the remaining two to integers...
cutDownData.drop(columns=['GCms', 'splittedGameClock'], inplace=True)
cutDownData[['GameClockMinute', 'GameClockSecond']] = cutDownData[['GameClockMinute', 'GameClockSecond']].astype(int)

# Add the Half column. Quarters 3 and 4 are the 2nd half..
cutDownData['Half'] = 1
cutDownData.loc[cutDownData.Quarter >= 3, 'Half'] = 2
cutDownData.head()

,GameId,PlayId,Team,X,Y,S,A,Dis,Orientation,Dir,...,Position,HomeTeamAbbr,VisitorTeamAbbr,Week,Stadium,Location,Turf,GameClockMinute,GameClockSecond,Half
0,2017090700,20170907000118,away,73.910004,34.840000,1.69,1.13,0.40,81.989998,177.179993,...,SS,NE,KC,1,Gillette Stadium,"Foxborough, MA",Field Turf,14,14,1
1,2017090700,20170907000118,away,74.669998,32.639999,0.42,1.35,0.01,27.610001,198.699997,...,DE,NE,KC,1,Gillette Stadium,"Foxborough, MA",Field Turf,14,14,1
2,2017090700,20170907000118,away,74.000000,33.200001,1.22,0.59,0.31,3.010000,202.729996,...,DE,NE,KC,1,Gillette Stadium,"Foxborough, MA",Field Turf,14,14,1
3,2017090700,20170907000118,away,71.459999,27.700001,0.42,0.54,0.02,359.769989,105.639999,...,ILB,NE,KC,1,Gillette Stadium,"Foxborough, MA",Field Turf,14,14,1
4,2017090700,20170907000118,away,69.320000,35.419998,1.82,2.43,0.16,12.630000,164.309998,...,FS,NE,KC,1,Gillette Stadium,"Foxborough, MA",Field Turf,14,14,1


### Player Height
In the data, the height of player is displayed as "feet-inches". We want to transform this into a float number i.e. "6-2" (6 feet 2 inches) becomes 6.1667 feet. The method is similar to the game clock, where we split on a delimiter. While we could use `apply()` to calculate the number, the function is generally very slow compared built-in element-wise operations (summing two columns using the "+" operator directly on the columns is much faster than calling `apply()` on each row). Thus, we'll split into two columns and combine them directly...

In [8]:
splittedHeight = pd.DataFrame(cutDownData.PlayerHeight.str.split('-').tolist(), index=cutDownData.index,
                             columns=['Feet', 'Inches']).astype(int)
cutDownData.PlayerHeight = splittedHeight.Feet + splittedHeight.Inches / 12
cutDownData.PlayerHeight.head()

0    6.00
1    6.25
2    6.25
3    6.25
4    6.00
Name: PlayerHeight, dtype: float64

Cool that was quick and snappy! The two columns **TimeHandoff** and **TimeSnap** have already been converted to timestamps thanks to `parse_dates()` in `read_csv()`. The last of bit of transformation is looking at the offensive and defensive personnel that is listed for each play.

### Offensive and Defensive Personnel
Personnel packages are a huge part of play design. Essentially, an offensive "package" mainly tells you how many running backs and tight ends the offense has on the field. Then, given there is (usually) one quarterback and 5 offensive linemen, you can calculate how many wide receivers there are. The "default" (subject to coach and team) is 11 (pronounce as the number) personnel, which is **one** running back and **one** tight end, which results in 3 wide recievers. You can have 12 personnel (two tight ends this time), 21 personnel (two RBs and 1 TE), etc. The personnel usually tells you the type of blocking that might happen. For example, if a short gain is needed (3rd and 1), the team might have 13 personnel, for the extra tight ends to provide extra blocking.

Defensive personnel work similarly, though there aren't fancy names for them, since the defense always reacts to what the offense does. However, we still include **defensive linemen** (DL), **linebackers** (LB), and cornerbacks/safeties, which gather into **defensive backs** (DB). More defensive backs means the defense is prioritizing speed over raw power and tanks and vice versa if there are more DLs. Linebackers provide a good middle ground, they though they often can't match-up one-on-one with running backs, as the latter are often too fast.

**EXTRA NOTE: It's quite possible we may never use this information due to having the physical characteristics of each player, but you may never know. Plus, it's good practice.**

#### Parsing Offensive Personnel
First, we'll deal with offensive personnel. Very rarely, sometimes we'll have a defensive player be in the personnel package, and maybe even sometimes **give him the ball**. For example, when Akiem Hicks scored a touchdown in the Bears Giants game of 2018 Week 13. However, this is extremely rare, and so we'll just package them into a column called "Defense Players".


In [9]:
def extractOffense(x):
    splittedOffense = [tuple(cat.strip().split(' ')) for cat in x.split(',')]
    # Values go RB, TE, WR, OL, QB, and Defense
    # Fill with default values...
    row = [1, 1, 3, 5, 1, 0]
    for num, position in splittedOffense:
        num = int(num)
        if position == 'RB':
            row[0] = num
        elif position == 'TE':
            row[1] = num
        elif position == 'WR':
            row[2] = num
        elif position == 'OL':
            row[3] = num
        elif position == 'QB':
            row[4] = num
        else:
            row[5] = num
    return row
offenseData = pd.DataFrame(cutDownData.OffensePersonnel.apply(extractOffense).values.tolist(), 
                           columns=['RB', 'TE', 'WR', 'OL', 'QB', 'DoO'], index=cutDownData.index, dtype=np.int16)

In [10]:
# Sanity check, make sure we have
# 11 players for each play...
np.all(offenseData.sum(axis=1) == 11)

True

Alright so we have the offensive breakdown for each play now. I used the `apply` function on the `Series` object itself, because it is much faster than using `apply` on the entire `DataFrame`. The **DoO** column stands for "Defense on Offense".

#### Parsing Defensive Personnel
The same method is used here, except the categories are now DL, LB, and DB, plus any extra offensive players.

In [11]:
def extractDefense(x):
    splittedDefense = [tuple(cat.strip().split(' ')) for cat in x.split(',')]
    # Values go DL, LB, DB, and Offense...
    row = [4, 3, 4, 0]
    for num, position in splittedDefense:
        num = int(num)
        if position == 'DL':
            row[0] = num
        elif position == 'LB':
            row[1] = num
        elif position == 'DB':
            row[2] = num
        else:
            row[3] = num
    return row
defenseData = pd.DataFrame(cutDownData.DefensePersonnel.apply(extractDefense).values.tolist(),
                          columns=['DL', 'LB', 'DB', 'OoD'], index=cutDownData.index, dtype=np.int16)

In [12]:
# Sanity check once more...
np.all(defenseData.sum(axis=1) == 11)

False

Uh oh! Looks like we have some discrepancy in that there was a play where there weren't 11 defensive players on the field. Let's investigate further.

In [13]:
cutDownData.loc[defenseData.sum(axis=1) != 11].shape

(22, 46)

Since each play has 22 rows (one for each player), it seems that there is only **one** play like this. Let's look at some clarifying information about the play.

In [14]:
cutDownData.loc[defenseData.sum(axis=1) != 11, 
                ['PlayId', 'Season', 'Week', 'Quarter', 'PossessionTeam', 'Yards', 'GameClockMinute', 'GameClockSecond']].iloc[0]

PlayId             20191006040465
Season                       2019
Week                            5
Quarter                         1
PossessionTeam                NYG
Yards                          11
GameClockMinute                 7
GameClockSecond                 0
Name: 575542, dtype: object

So it seems it was a Giants game in Week 5 of the 2019 Season at the 7:00 mark in the 1st quarter, where the rusher rushed for 11 yards. Using Pro-Football-Reference, I've pinpointed the play [here](https://www.pro-football-reference.com/play-index/play_finder.cgi?request=1&match=summary_all&year_min=2019&year_max=2019&team_id=nyg&game_type=R&game_num_min=0&game_num_max=99&week_num_min=5&week_num_max=5&quarter%5B%5D=1&minutes_max=15&seconds_max=0&minutes_min=0&seconds_min=0&down%5B%5D=0&down%5B%5D=1&down%5B%5D=2&down%5B%5D=3&down%5B%5D=4&field_pos_min_field=team&field_pos_max_field=team&end_field_pos_min_field=team&end_field_pos_max_field=team&type%5B%5D=PASS&type%5B%5D=RUSH&type%5B%5D=PUNT&type%5B%5D=KOFF&type%5B%5D=ONSD&type%5B%5D=FG&type%5B%5D=XP&type%5B%5D=2PC&no_play=N&turnover_type%5B%5D=interception&turnover_type%5B%5D=fumble&score_type%5B%5D=touchdown&score_type%5B%5D=field_goal&score_type%5B%5D=safety&rush_direction%5B%5D=LE&rush_direction%5B%5D=LT&rush_direction%5B%5D=LG&rush_direction%5B%5D=M&rush_direction%5B%5D=RG&rush_direction%5B%5D=RT&rush_direction%5B%5D=RE&pass_location%5B%5D=SL&pass_location%5B%5D=SM&pass_location%5B%5D=SR&pass_location%5B%5D=DL&pass_location%5B%5D=DM&pass_location%5B%5D=DR&order_by=yards). Make sure to scroll down to "Individual Plays". You can see that the description of the play is "*Wayne Gallman up the middle for 11 yards (tackle by Anthony Harris and Eric Kendricks) Penalty on MIN: Defensive Too Many Men on Field (Declined)*". **It was a penalty!** The Vikings had too many men on the field, but the play wasn't called dead, so the result ended up going into the books. In this case, the Vikings were probably confused on what was going on, and so were *not* on high alert when the play occurred. Thus explains why they let a 11-yard play take place when they had 12 men on the field to stop it.

So what should we do? Since it's just one play out of the 30 thousand we have, we can just drop it.

#### Adding to DataFrame
Let's add our extracted offense and defense player counts to the data frame, making sure to drop that one play.

In [15]:
cutDownData = pd.concat([cutDownData, offenseData, defenseData], axis=1)
indicesToDrop = cutDownData.loc[cutDownData.PlayId == 20191006040465].index
cutDownData.drop(index=indicesToDrop, inplace=True)
cutDownData.shape

(673410, 56)

Okay, that covers all column transformations we will do here!